In [1]:
!pip install pennylane --quiet

You should consider upgrading via the '/home/favour_nerrise/Al-HaQ/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import sys
import torch

# sys.path.insert(1, "/home/favour_nerrise/Al-HaQ")
from src.preprocessing import FakeTweetsDataset, create_mini_batch
from torch.utils.data import DataLoader
from torch import nn

import models.qnn as qnn

from transformers import AutoTokenizer, XLNetForSequenceClassification
from IPython.display import display, clear_output
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import os

os.environ["CURL_CA_BUNDLE"] = ""
os.environ["REQUESTS_CA_BUNDLE"] = ""

In [3]:
BATCH_SIZE = 56

proxy = {"http": "http://127.0.0.1:7890", "https": "http://127.0.0.1:7890"}
MODEL_NAME = "xlnet-base-cased"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

cla_model = XLNetForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

qcnn = qnn.QCNN(4)

data_path = "datasets/twitter_dataset"
trainset = FakeTweetsDataset("train", tokenizer=tokenizer, path=data_path)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cla_model.to(device)
qcnn.to(device)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


QCNN(
  (qcnn_layer): <Quantum Torch Layer: func=qcnn4>
)

In [5]:
cla_model.train()
qcnn.train()

optimizer = torch.optim.AdamW(
    [{"params": cla_model.parameters()}, {"params": qcnn.parameters()}], lr=1e-3
)

loss_func = nn.CrossEntropyLoss()

NUM_EPOCHS = 50
print(f"data length:{trainset.len}")

data length:114069


In [6]:
for epoch in range(NUM_EPOCHS):
    train_loss = 0.0
    train_acc = 0.0

    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}]")

    for batch_idx, data in enumerate(trainloader):

        tokens_tensors, segments_tensors, masks_tensors, labels = [
            t.to(device) for t in data
        ]

        # labels = labels.type(torch.LongTensor).to(device)

        optimizer.zero_grad()
        q_input = cla_model(
            input_ids=tokens_tensors,
            token_type_ids=segments_tensors,
            attention_mask=masks_tensors,
        )[0].to(device)

        outputs = qcnn(q_input)

        loss = loss_func(outputs, labels)

        loss.backward()
        optimizer.step()

        pred = torch.argmax(outputs, dim=1)
        train_acc = accuracy_score(pred.cpu().tolist(), labels.cpu().tolist())

        train_loss += loss.item()

    print(f"acc={train_acc}, loss={train_loss / (batch_idx + 1):.2f}")

Epoch [1/50]
target
0    45139
1    68930
dtype: int64


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0, cpu!

In [ ]:
torch.save(cla_model, "results/xlnet_twitter.pth")
torch.save(qcnn.state_dict(), "results/qcnn_twitter_tx.pth")